In [1]:
import torch
from transformers import BertTokenizer, BertForMaskedLM, DataCollatorForLanguageModeling
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
import accelerate

C:\ProgramData\miniconda3\envs\tensorflow-gpu\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\miniconda3\envs\tensorflow-gpu\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\ProgramData\miniconda3\envs\tensorflow-gpu\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
tf_device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'
print(tf_device)

NameError: name 'tf' is not defined

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'NVIDIA RTX A5000'

In [2]:
# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-german-cased")

In [11]:
def load_and_prepare_dataset(dataset_path):
    # Load your custom dataset
    dataset = load_dataset('text', data_files={'train': dataset_path})

    # Tokenize the text
    def tokenize_function(examples):
        return tokenizer(examples["text"], return_special_tokens_mask=True, truncation=True, padding="max_length", max_length=512)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    return tokenized_datasets

In [12]:
dataset_path = '../dataset/clean_text_data.txt'
tokenized_datasets = load_and_prepare_dataset(dataset_path)

Map:   0%|          | 0/76697 [00:00<?, ? examples/s]

In [13]:
# Data collator used for dynamic masking
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [14]:
model = BertForMaskedLM.from_pretrained("bert-base-german-cased")
model.to(device)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [15]:
training_args = TrainingArguments(
    output_dir="./bert-german-checkpoints",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
)

In [17]:
trainer.train()

Step,Training Loss
500,2.460400
1000,2.082900
1500,1.965700
2000,1.891200
2500,1.811600
3000,1.725900
3500,1.710000
4000,1.662900
4500,1.606400
5000,1.587400


TrainOutput(global_step=7191, training_loss=1.7504446119579813, metrics={'train_runtime': 5350.4964, 'train_samples_per_second': 43.004, 'train_steps_per_second': 1.344, 'total_flos': 6.056068961171866e+16, 'train_loss': 1.7504446119579813, 'epoch': 3.0})

In [18]:
model.save_pretrained("./bert-german-science")

In [10]:
from transformers import *
model = TFBertForSequenceClassification.from_pretrained('../models/G-SciEdBERT_model_combined')

loading configuration file ../models/G-SciEdBERT_model_combined\config.json
Model config BertConfig {
  "_name_or_path": "../models/G-SciEdBert",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformer

In [5]:
!huggingface-cli login

^C


In [7]:
model = BertForMaskedLM.from_pretrained('../models/G-SciEdBert')

loading configuration file ../models/G-SciEdBert\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-german-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.37.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

loading weights file ../models/G-SciEdBert\pytorch_model.bin
Generate config GenerationConfig {
  "pad_token_id": 0
}

All model checkpoint weights were used when initializing BertForMaskedLM.

All the weights of BertForMaskedLM were initialized from the model checkpoint at ../mo

In [13]:
tokenizer.push_to_hub("ai4stem-uga/G-SciEdBERT-scoring")

README.md:   0%|          | 0.00/874 [00:00<?, ?B/s]

tokenizer config file saved in C:\Users\el44163\AppData\Local\Temp\tmprcrej9vp\tokenizer_config.json
Special tokens file saved in C:\Users\el44163\AppData\Local\Temp\tmprcrej9vp\special_tokens_map.json
Uploading the following files to ai4stem-uga/G-SciEdBERT-scoring: README.md,special_tokens_map.json,tokenizer_config.json,vocab.txt


CommitInfo(commit_url='https://huggingface.co/ai4stem-uga/G-SciEdBERT-scoring/commit/00bff4898accba0e5b080ca80e6532c3422277fc', commit_message='Upload tokenizer', commit_description='', oid='00bff4898accba0e5b080ca80e6532c3422277fc', pr_url=None, pr_revision=None, pr_num=None)

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased', do_lower_case=True) # tokenizer

loading file vocab.txt from cache at C:\Users\el44163\.cache\huggingface\hub\models--bert-base-german-cased\snapshots\55717ce5b34282a966d7a6e6fc26a67ae4312dcd\vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\el44163\.cache\huggingface\hub\models--bert-base-german-cased\snapshots\55717ce5b34282a966d7a6e6fc26a67ae4312dcd\tokenizer_config.json
loading file tokenizer.json from cache at C:\Users\el44163\.cache\huggingface\hub\models--bert-base-german-cased\snapshots\55717ce5b34282a966d7a6e6fc26a67ae4312dcd\tokenizer.json
loading configuration file config.json from cache at C:\Users\el44163\.cache\huggingface\hub\models--bert-base-german-cased\snapshots\55717ce5b34282a966d7a6e6fc26a67ae4312dcd\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-german-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "class

In [11]:
model.push_to_hub("ai4stem-uga/G-SciEdBERT-scoring")

Configuration saved in C:\Users\el44163\AppData\Local\Temp\tmpkicxb2sz\config.json
Model weights saved in C:\Users\el44163\AppData\Local\Temp\tmpkicxb2sz\tf_model.h5
Uploading the following files to ai4stem-uga/G-SciEdBERT-scoring: config.json,README.md,tf_model.h5


tf_model.h5:   0%|          | 0.00/437M [00:00<?, ?B/s]

C:\ProgramData\miniconda3\envs\tensorflow-gpu\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\miniconda3\envs\tensorflow-gpu\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\ProgramData\miniconda3\envs\tensorflow-gpu\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
Traceback (most recent call last):
  File "C:\ProgramData\miniconda3\envs\tensorflow-gpu\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\ProgramData\miniconda3\envs\tensorflow-gpu\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\miniconda3\envs\tensorflow-gpu\Scripts\transformers-cli.exe\__main__.py", line 4, in <module>
  File "C:\ProgramData\miniconda3\envs\tensorflow-gpu\lib\site-packages\transformers\commands\transformers_cli.py", line 26, i